<a href="https://colab.research.google.com/github/vrlnarayana/capstone-gl/blob/main/SS_Content_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project : Group 4 : Song Recommendation through content based filtering

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from google.colab import files
from scipy.sparse import csr_matrix
import warnings; warnings.simplefilter('ignore')

# Remove scientific notations and display numbers with 2 decimal points instead
pd.options.display.float_format = '{:,.2f}'.format   

plt.rcParams["figure.figsize"] = [20, 6]  # width, height
plt.rcParams['xtick.direction'] = 'out'

**READ SONG DATA**

In [3]:
song_data_df =pd.read_csv(r'/content/drive/My Drive/songs_with_genre_year_location.csv')

In [4]:
song_data_df.duplicated(subset=['track_id']).value_counts()

False    112154
True        209
dtype: int64

There are 209 duplicate records in the song dataframe

In [5]:
song_data_df = song_data_df.drop_duplicates(subset=['track_id'])

**READ Bags of words, extracting them from the train dataset "**

In [6]:
#import file
lyrics = pd.read_table(r'/content/drive/My Drive/mxm_dataset_train.txt', error_bad_lines=False)

#change name of the column
lyrics.columns = ['Raw_Training']

# take the bag of word to use it later
words_train = lyrics.iloc[16]

#drop useless rows
lyrics=lyrics[17:].copy()

# get TrackID, MxMID and lyrics and put them separated columns
def sortdata(x):
    splitted = x['Raw_Training'].split(',')
    x['Tid']=splitted[0]
    #x['MXM_Tid']=splitted[1]
    x['words_freq']=splitted[2:]
    return x

#Apply the function to every column
lyrics = lyrics.apply(sortdata,axis=1)
lyrics = lyrics[['Tid','words_freq']]

In [7]:
lyrics

,Tid,words_freq
17,TRAAAAV128F421A322,"[1:6, 2:4, 3:2, 4:2, 5:5, 6:3, 7:1, 8:1, 11:1,..."
18,TRAAABD128F429CF47,"[1:10, 3:17, 4:8, 5:2, 6:2, 7:1, 8:3, 9:2, 10:..."
19,TRAAAED128E0783FAB,"[1:28, 2:15, 3:2, 4:12, 5:22, 6:2, 7:2, 8:4, 9..."
20,TRAAAEF128F4273421,"[1:5, 2:4, 3:3, 4:2, 5:1, 6:11, 9:4, 12:9, 13:..."
21,TRAAAEW128F42930C0,"[1:4, 4:5, 5:7, 6:2, 7:4, 9:1, 10:1, 11:9, 15:..."
...,...,...
210531,TRZZZWS128F429CF87,"[6:1, 24:9, 38:7, 42:1, 77:4, 90:1, 112:5, 114..."
210532,TRZZZXA128F428ED56,"[1:1, 2:13, 3:6, 4:5, 5:4, 6:6, 9:4, 10:4, 12:..."
210533,TRZZZXV128F4289747,"[1:13, 2:3, 3:17, 4:5, 5:9, 7:8, 8:2, 9:1, 11:..."
210534,TRZZZYV128F92E996D,"[1:10, 2:6, 3:20, 5:2, 7:30, 8:1, 9:6, 10:3, 1..."


There are no duplicates inn the lyrics dataframe

In [8]:
len(song_data_df)

112154

In [9]:
len(lyrics['Tid'].unique())

210519

In [10]:
song_data_bow_df =pd.merge(song_data_df,lyrics,left_on='track_id', right_on='Tid',how='left')

In [11]:
song_data_bow_df

,song_id,title,release,artist_name,year,track_id,artist_id,duration,artist_familiarity,artist_hotttnesss,lat,long,location,genre,Tid,words_freq
0,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,ARGEKB01187FB50750,138.97,0.64,0.44,55.86,-4.24,"Glasgow, Scotland",Electronic,NaN,NaN
1,SOEYRFT12AB018936C,2 Da Beat Ch'yall,Da Bomb,Kris Kross,1993,TRMMMBB12903CB7D21,AR3Z9WY1187FB4CDC2,221.20,0.59,0.40,38.90,-77.03,Washington DC,Rap,NaN,NaN
2,SOBARPM12A8C133DFF,(Looking For) The Heart Of Saturday,Cover Girl,Shawn Colvin,1994,TRMMMUT128F42646E8,ARDW5AW1187FB55708,216.48,0.69,0.45,42.79,-96.93,"Vermillion, SD",Pop_Rock,TRMMMUT128F42646E8,"[2:36, 3:25, 4:2, 5:8, 6:2, 7:1, 8:6, 10:8, 12..."
3,SOKOVRQ12A8C142811,Ethos of Coercion,Descend Into Depravity,Dying Fetus,2009,TRMMMQY128F92F0EA3,ARGWPP11187B9AEF43,196.02,0.73,0.51,38.82,-75.92,Maryland,Pop_Rock,NaN,NaN
4,SOOLRHW12A8C142643,All of the same blood,Violent revolution,Kreator,2001,TRMMMKQ128F92EBCB5,AR79L0D1187FB3AFB6,372.43,0.74,0.47,51.45,7.01,Essen,Pop_Rock,TRMMMKQ128F92EBCB5,"[2:12, 3:2, 4:3, 5:4, 10:2, 12:1, 13:9, 15:2, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112149,SOSCSAQ12A6D4FC60F,Papa's Medicine Show,Local Gentry,Bobbie Gentry,1968,TRYYKQP128F14A3A6D,ARQXAUH1187B9BA010,229.98,0.58,0.38,33.91,-88.96,"Chickasaw County, MS",Country,NaN,NaN
112150,SOCZDVJ12AB0184A17,Dalla finestra,I Grandi Successi: Pierangelo Bertoli,Pierangelo Bertoli,1984,TRYYKFJ128F93465A0,ARX8BQT1187B9AB472,275.83,0.48,0.43,44.54,10.78,"Sassuolo, Italy",Pop_Rock,NaN,NaN
112151,SOFGQKB12AB017D895,Feeding The Demons,Serpent Smiles And Killer Eyes,Hatesphere,2007,TRYYYAF128F930C01F,ARLPWXD1187FB4CB6A,266.63,0.65,0.41,55.68,12.57,Copenhagen/Aarhus - Denmark,Pop_Rock,TRYYYAF128F930C01F,"[1:5, 2:2, 4:2, 6:1, 7:7, 8:5, 11:7, 13:1, 15:..."
112152,SONVYTH12AB0182C89,So Sad About Us / Brrr,Scoop,Pete Townshend,1983,TRYYYOI128F933F153,ARLVX371187B9AF852,284.53,0.61,0.45,51.49,-0.26,"Chiswick, Greater London, England",Pop_Rock,NaN,NaN


In [12]:
song_data_bow_df = song_data_bow_df[song_data_bow_df['words_freq'].notna()]

In [13]:
#Displaying the results
print(song_data_bow_df.shape)
display(song_data_bow_df.head())

(40628, 16)


,song_id,title,release,artist_name,year,track_id,artist_id,duration,artist_familiarity,artist_hotttnesss,lat,long,location,genre,Tid,words_freq
2,SOBARPM12A8C133DFF,(Looking For) The Heart Of Saturday,Cover Girl,Shawn Colvin,1994,TRMMMUT128F42646E8,ARDW5AW1187FB55708,216.48,0.69,0.45,42.79,-96.93,"Vermillion, SD",Pop_Rock,TRMMMUT128F42646E8,"[2:36, 3:25, 4:2, 5:8, 6:2, 7:1, 8:6, 10:8, 12..."
4,SOOLRHW12A8C142643,All of the same blood,Violent revolution,Kreator,2001,TRMMMKQ128F92EBCB5,AR79L0D1187FB3AFB6,372.43,0.74,0.47,51.45,7.01,Essen,Pop_Rock,TRMMMKQ128F92EBCB5,"[2:12, 3:2, 4:3, 5:4, 10:2, 12:1, 13:9, 15:2, ..."
9,SOVOIOQ12AC4688FAC,The Rest of the Night,The Wind,Warren Zevon,2003,TRMMWNY12903CF28D2,ARUQFUJ1187FB42AEC,279.30,0.69,0.47,41.88,-87.63,"Chicago, IL",Pop_Rock,TRMMWNY12903CF28D2,"[1:1, 2:31, 3:2, 4:1, 5:2, 6:1, 7:4, 8:2, 10:1..."
13,SOGXPYA12A58A769A7,Rock Can't Last Forever (You Hate Me Hate Mus...,You Make Me Hate Music,Fine China,2002,TRMMGDQ128F4277AE7,AR6KJQA1187FB3EF30,295.94,0.49,0.36,33.45,-112.08,"Phoenix, Arizona",Pop_Rock,TRMMGDQ128F4277AE7,"[1:5, 2:1, 3:4, 4:5, 5:6, 6:3, 7:2, 8:3, 9:3, ..."
21,SOWSOVJ12A8C13A8C1,Zuviel Hitze,Einzelhaft,Falco,1982,TRMMHVN128F428E67E,AR5DXUU1187B9B0E42,274.02,0.67,0.46,48.20,16.37,"Vienna, Austria",Pop_Rock,TRMMHVN128F428E67E,"[3:1, 10:5, 14:1, 20:1, 28:2, 32:1, 33:1, 73:1..."



**From generic bags of words to lyrics**

In [14]:
#get the word data
bag_of_words = words_train
# clean the data and split it to create a list of 5000 words
bag_of_words = bag_of_words.str.replace('%','')
bag_of_words = bag_of_words.str.split(',')

display(bag_of_words.head())


Raw_Training    [i, the, you, to, and, a, me, it, not, in, my,...
Name: 16, dtype: object

In [15]:
#Defining a function
def create_text(words_freq):
    #create the final list of all words
    list_words=''
    #iterate over every id of words
    for compteur in words_freq:
        
        word = bag_of_words[0][int(compteur.split(':')[0])-1]
        times = int(compteur.split(':')[1])
        
        #Separating every word with a space to be able to work on it with librairies during part 2
        for i in range(times):
            list_words += ' ' + word + ' '
    return list_words

In [17]:
#Testing the function
print(create_text(lyrics.iloc[0]['words_freq']))

 i  i  i  i  i  i  the  the  the  the  you  you  to  to  and  and  and  and  and  a  a  a  me  it  my  is  is  of  of  of  your  that  are  are  we  we  am  am  will  will  for  for  for  for  be  have  have  so  this  like  like  de  up  was  was  if  got  would  been  these  these  seem  someon  understand  pass  river  met  piec  damn  worth  flesh  grace  poor  poor  somehow  ignor  passion  tide  season  seed  resist  order  order  piti  fashion  grant  captur  captur  ici  soil  patienc  social  social  highest  highest  slice  leaf  lifeless  arrang  wilder  shark  devast  element 


In [18]:
song_data_bow_df['text'] = song_data_bow_df['words_freq'].apply(create_text)

In [19]:
sample_song_df = song_data_bow_df.head(1000)

In [20]:
sample_song_df = sample_song_df.set_index('title')
sample_song_df.head(4)

,song_id,release,artist_name,year,track_id,artist_id,duration,artist_familiarity,artist_hotttnesss,lat,long,location,genre,Tid,words_freq,text
title,,,,,,,,,,,,,,,,
(Looking For) The Heart Of Saturday,SOBARPM12A8C133DFF,Cover Girl,Shawn Colvin,1994,TRMMMUT128F42646E8,ARDW5AW1187FB55708,216.48,0.69,0.45,42.79,-96.93,"Vermillion, SD",Pop_Rock,TRMMMUT128F42646E8,"[2:36, 3:25, 4:2, 5:8, 6:2, 7:1, 8:6, 10:8, 12...",the the the the the the the the the ...
All of the same blood,SOOLRHW12A8C142643,Violent revolution,Kreator,2001,TRMMMKQ128F92EBCB5,AR79L0D1187FB3AFB6,372.43,0.74,0.47,51.45,7.01,Essen,Pop_Rock,TRMMMKQ128F92EBCB5,"[2:12, 3:2, 4:3, 5:4, 10:2, 12:1, 13:9, 15:2, ...",the the the the the the the the the ...
The Rest of the Night,SOVOIOQ12AC4688FAC,The Wind,Warren Zevon,2003,TRMMWNY12903CF28D2,ARUQFUJ1187FB42AEC,279.30,0.69,0.47,41.88,-87.63,"Chicago, IL",Pop_Rock,TRMMWNY12903CF28D2,"[1:1, 2:31, 3:2, 4:1, 5:2, 6:1, 7:4, 8:2, 10:1...",i the the the the the the the the th...
Rock Can't Last Forever (You Hate Me Hate Music Album Version),SOGXPYA12A58A769A7,You Make Me Hate Music,Fine China,2002,TRMMGDQ128F4277AE7,AR6KJQA1187FB3EF30,295.94,0.49,0.36,33.45,-112.08,"Phoenix, Arizona",Pop_Rock,TRMMGDQ128F4277AE7,"[1:5, 2:1, 3:4, 4:5, 5:6, 6:3, 7:2, 8:3, 9:3, ...",i i i i i the you you you you to t...


In [21]:
indices = pd.Series(sample_song_df.index)
indices[:5]

0                  (Looking For) The Heart Of Saturday
1                                All of the same blood
2                                The Rest of the Night
3    Rock Can't Last Forever  (You Hate Me Hate Mus...
4                                         Zuviel Hitze
Name: title, dtype: object

 
Comments data processing:
As it is noticeable through each step, we loose data every time we merge datasets. We chose this approach because we only want to deal with complete information in order to be coherent. We want to compare parameters between items and we believe that the analysis is less relevant if we consider a larger dataset that contains data incomplete data.

In [22]:
#!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [24]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z]"," ",text.lower())
    tokens = word_tokenize(text)
    words = [word for word in tokens if word not in stopwords.words("english")]
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]

    return text_lems

In [25]:
tfidf = TfidfVectorizer(tokenizer=tokenize)
song_tfidf = tfidf.fit_transform(sample_song_df['text'].values).toarray()

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
similar_songs = cosine_similarity(song_tfidf, song_tfidf)
similar_songs

array([[1.        , 0.01841732, 0.06638799, ..., 0.02476007, 0.02001813,
        0.02268992],
       [0.01841732, 1.        , 0.        , ..., 0.01377702, 0.05526707,
        0.03081957],
       [0.06638799, 0.        , 1.        , ..., 0.00878402, 0.00638369,
        0.00752986],
       ...,
       [0.02476007, 0.01377702, 0.00878402, ..., 1.        , 0.00816878,
        0.09504809],
       [0.02001813, 0.05526707, 0.00638369, ..., 0.00816878, 1.        ,
        0.03236887],
       [0.02268992, 0.03081957, 0.00752986, ..., 0.09504809, 0.03236887,
        1.        ]])

In [28]:
# function that takes in song title as input and returns the top 10 recommended songs
def recommendations(title, similar_songs):
    
    recommended_songs = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(similar_songs[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_songs.append(list(sample_song_df.index)[i])
        
    return recommended_songs

In [29]:
recommendations('The Rest of the Night', similar_songs)

[71, 300, 283, 677, 986, 428, 733, 208, 848, 812]


['Co Star (Album Version)',
 'The Get Go',
 'Undeclared',
 'Pretty',
 "It's Love",
 "I'll Be Your Lampshade",
 'Nothing From Something (Album Version)',
 "She's Got To Know  (LP Version)",
 'Show Me The Money',
 'Rest Stop Cleaning']

In [30]:
print(sample_song_df.index[71])
print(sample_song_df.iloc[71]['genre'])

Co Star (Album Version)
RnB


In [32]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 7.5MB/s 


In [33]:
import nltk.sentiment.sentiment_analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [34]:
analyser = SentimentIntensityAnalyzer()

In [35]:
sample_song_df.reset_index()

,title,song_id,release,artist_name,year,track_id,artist_id,duration,artist_familiarity,artist_hotttnesss,lat,long,location,genre,Tid,words_freq,text
0,(Looking For) The Heart Of Saturday,SOBARPM12A8C133DFF,Cover Girl,Shawn Colvin,1994,TRMMMUT128F42646E8,ARDW5AW1187FB55708,216.48,0.69,0.45,42.79,-96.93,"Vermillion, SD",Pop_Rock,TRMMMUT128F42646E8,"[2:36, 3:25, 4:2, 5:8, 6:2, 7:1, 8:6, 10:8, 12...",the the the the the the the the the ...
1,All of the same blood,SOOLRHW12A8C142643,Violent revolution,Kreator,2001,TRMMMKQ128F92EBCB5,AR79L0D1187FB3AFB6,372.43,0.74,0.47,51.45,7.01,Essen,Pop_Rock,TRMMMKQ128F92EBCB5,"[2:12, 3:2, 4:3, 5:4, 10:2, 12:1, 13:9, 15:2, ...",the the the the the the the the the ...
2,The Rest of the Night,SOVOIOQ12AC4688FAC,The Wind,Warren Zevon,2003,TRMMWNY12903CF28D2,ARUQFUJ1187FB42AEC,279.30,0.69,0.47,41.88,-87.63,"Chicago, IL",Pop_Rock,TRMMWNY12903CF28D2,"[1:1, 2:31, 3:2, 4:1, 5:2, 6:1, 7:4, 8:2, 10:1...",i the the the the the the the the th...
3,Rock Can't Last Forever (You Hate Me Hate Mus...,SOGXPYA12A58A769A7,You Make Me Hate Music,Fine China,2002,TRMMGDQ128F4277AE7,AR6KJQA1187FB3EF30,295.94,0.49,0.36,33.45,-112.08,"Phoenix, Arizona",Pop_Rock,TRMMGDQ128F4277AE7,"[1:5, 2:1, 3:4, 4:5, 5:6, 6:3, 7:2, 8:3, 9:3, ...",i i i i i the you you you you to t...
4,Zuviel Hitze,SOWSOVJ12A8C13A8C1,Einzelhaft,Falco,1982,TRMMHVN128F428E67E,AR5DXUU1187B9B0E42,274.02,0.67,0.46,48.20,16.37,"Vienna, Austria",Pop_Rock,TRMMHVN128F428E67E,"[3:1, 10:5, 14:1, 20:1, 28:2, 32:1, 33:1, 73:1...",you in in in in in your am so so wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,New Breed,SOTVDJD12A58A7DACE,Negative Outlook,Sworn Enemy,2004,TRMLHXD128F422872B,ARLL29O1187B996028,205.11,0.68,0.37,40.70,-73.83,"Queens, NY",Pop_Rock,TRMLHXD128F422872B,"[2:18, 3:1, 4:7, 5:4, 6:4, 8:1, 9:11, 10:3, 12...",the the the the the the the the the ...
996,Lightning Rod (Album Version),SOLDAZZ12A6D4F8DA5,Ganging Up On The Sun,Guster,2006,TRMLHNZ128F1480EE8,AR57LEA1187B9AD137,175.57,0.73,0.54,42.41,-71.13,"West Somerville, Massachusetts",Pop_Rock,TRMLHNZ128F1480EE8,"[1:3, 2:4, 5:2, 6:2, 10:2, 12:1, 17:2, 18:3, 2...",i i i the the the the and and a a ...
997,Houston We Have a Problem,SOBFQDS12A8C13A1D1,Munki Brain,The Queers,2007,TRMLCSC128F4270F6C,ARIEKZA1187B98FED4,129.07,0.68,0.40,43.07,-70.77,"Portsmouth, New Hampshire",Pop_Rock,TRMLCSC128F4270F6C,"[1:11, 2:7, 3:1, 4:3, 5:11, 6:5, 8:2, 9:2, 10:...",i i i i i i i i i i i the the th...
998,Sweet Charity And Adoration,SOMAVEE12A8C13C741,Diary Of A Hollow Horse,China Crisis,1989,TRMLCUL128F427D6BB,ARGJ84O1187B98CD2C,291.79,0.61,0.39,53.48,-2.88,"Kirkby, Merseyside, England",Pop_Rock,TRMLCUL128F427D6BB,"[1:5, 2:2, 3:23, 4:1, 5:1, 6:3, 7:1, 9:1, 10:5...",i i i i i the the you you you you ...


In [36]:

#Applying the polarity analysis for the bags of words
for i in sample_song_df.index:
    sample_song_df.at[i, 'Polarity_score'] = analyser.polarity_scores(sample_song_df.at[i, 'text'])['compound']
    
#Sorting values to have a meaningfull plot
sample_song_df = sample_song_df.sort_values('Polarity_score', ascending=False)

In [38]:
sample_song_df['Polarity_score'].unique()


array([ 0.9998,  0.9997,  0.9994,  0.9993,  0.9992,  0.9991,  0.9989,
        0.9988,  0.9987,  0.9986,  0.9985,  0.9984,  0.9983,  0.9982,
        0.9981,  0.998 ,  0.9979,  0.9978,  0.9977,  0.9976,  0.9975,
        0.9973,  0.9972,  0.9971,  0.997 ,  0.9969,  0.9966,  0.9965,
        0.9964,  0.9963,  0.9962,  0.9961,  0.996 ,  0.9959,  0.9957,
        0.9956,  0.9954,  0.9953,  0.9951,  0.995 ,  0.9949,  0.9948,
        0.9946,  0.9945,  0.9944,  0.9943,  0.9942,  0.9939,  0.9938,
        0.9936,  0.9935,  0.9933,  0.9931,  0.993 ,  0.9929,  0.9928,
        0.9926,  0.9925,  0.9924,  0.9922,  0.992 ,  0.9917,  0.9915,
        0.9914,  0.9913,  0.9912,  0.9911,  0.991 ,  0.9908,  0.9904,
        0.9902,  0.9899,  0.9897,  0.9896,  0.9895,  0.9892,  0.9888,
        0.9885,  0.9881,  0.988 ,  0.9879,  0.9877,  0.9875,  0.9874,
        0.9872,  0.9871,  0.987 ,  0.9869,  0.9868,  0.9867,  0.9866,
        0.9865,  0.9861,  0.9858,  0.9856,  0.9851,  0.9842,  0.9841,
        0.984 ,  0.9